In [ ]:
import sqlalchemy as sq

engine = sq.create_engine("postgresql://localhost/crime")
conn = engine.connect()

In [14]:
statement = """select ni.incident_date, ni.incident_hour, a.county_name, no.incident_id, t.offense_code, t.offense_name, nlt.location_name, nwt.weapon_name, nwt.weapon_code, nbm.bias_id from nibrs_incident ni
                JOIN agencies a on ni.agency_id = a.agency_id
                JOIN nibrs_offender no on ni.incident_id = no.incident_id
                JOIN nibrs_offense n on ni.incident_id = n.incident_id
                JOIN nibrs_location_type nlt on n.location_id = nlt.location_id
                JOIN nibrs_offense_type t on n.offense_code = t.offense_code
                JOIN nibrs_weapon nw on n.offense_id = nw.offense_id
                JOIN nibrs_weapon_type nwt on nw.weapon_id = nwt.weapon_id
                JOIN nibrs_bias_motivation nbm on n.offense_id = nbm.offense_id;"""
result = conn.execute(statement)

In [ ]:
SEX_CRIME_CODE = ["11", "36", "90H"]            # rape, sodomy, SA with object, fondling, incest, stat. rape, peeping tom - offense_code
FIREARM_CODE   = ["11", "12", "13", "14", "15"] # NIBRS_WEAPON_TYPE, firearm, handgun, rifle, shotgun, other firearm - weapon_code
HATE_CODE      = "88"                           # if result contains this number, it is NOT a hate crime - bias_id

def create_incident_time_dict(incident: dict) -> dict:
    time_dict = dict()
    print(incident)
    time_dict["incident_hour"] = incident["incident_hour"]
    time_dict["incident_day"] = incident["incident_date"].day
    time_dict["incident_month"] = incident["incident_date"].month
    time_dict["incident_year"] = incident["incident_date"].year
    return time_dict

def awful_array_string_checker(arr1, string) -> bool:
    for e in arr1:
        if e in string:
            return True
    return False

def check_codes(incident: dict) -> dict:
    incident['gun_violence'] = False
    incident['sex_crime']    = False
    incident['hate_crime']   = False
    if awful_array_string_checker(incident['offense_code'], SEX_CRIME_CODE):
        incident

res_dict = dict()
i = 0
for u in result.all():
    incident_dict = dict(u)

    finished_incident_dict = dict()
    time_dict = create_incident_time_dict(incident_dict)
    finished_incident_dict["incident_time"] = time_dict

    res_dict[i] = finished_incident_dict
    i+=1



In [ ]:
import json
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(res_dict, f, ensure_ascii=False, indent=4)